In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from analyzer.logic import load_contacts, row_to_prospect, plot_booking_rate, total_touches_analysis
from analyzer.search import ProspectSearcher

emails_per_person = 4.419
sequences_seen = 1.202

df = load_contacts("cleaned-data-main-seq-25.csv")
df.head()

,Record ID,First,Last,Name,Currently In Sequence,Job Title,Company,State,Number of Sequences Enrolled,Number of Sales Activities,Create Date,Last Sequence Enrolled,Last Sequence Ended Date,Last Contacted,Last Engagement Date,Meeting Booked
0,2.816631e+10,Michael,Reese,Michael Reese,False,"Head Senior VP, Corporate Real Estate & Facili...",Customers Bank,Pennsylvania,3,37,6/7/2024 14:09,269668391,3/7/2025 14:23,4/2/2025 11:15,4/16/2025 16:35,1
1,4.373301e+06,Kris,Weir,Kris Weir,False,"Space Planning Director, Operations",Questrom School of Business,Massachusetts,8,37,3/7/2022 9:28,279559810,7/9/2025 12:33,10/20/2025 15:00,10/16/2025 12:57,1
2,3.323551e+06,Scott,Albert,Scott Albert,False,Vice President of Facilities,Slippery Rock University,Pennsylvania,14,67,10/18/2021 9:14,279109668,7/30/2025 14:25,10/15/2025 10:20,10/15/2025 12:04,1
3,1.446120e+11,Amy,Ciba,Amy Ciba,False,"Senior Director, Global Workplaces",Meltwater,Stockholms län,1,9,8/6/2025 15:11,269580465,8/6/2025 16:46,9/4/2025 12:54,8/17/2025 14:52,1
4,3.000951e+06,Jessica,Davenport,Jessica Davenport,False,Director Campus Planning & Design,Salt Lake Community College,Utah,8,74,7/19/2021 17:23,279559810,7/9/2025 10:06,9/17/2025 15:39,7/15/2025 10:35,1


In [18]:
# Check significance between number of sequences enrolled and meeting booked rate
# Results - 7 sequences were statistically significant in a good way

amt_of_sequences = (
    df.groupby("Number of Sequences Enrolled")["Meeting Booked"]
        .agg(["count", "sum"])
        .reset_index()
    )
amt_of_sequences["Rate"] = amt_of_sequences["sum"] / amt_of_sequences["count"] * 100

total_success = (df["Meeting Booked"] != 0).sum()
total_contacts = len(df)
overall_rate = total_success / total_contacts * 100
print(f"Overall booking rate: {overall_rate}.3%")

Overall booking rate: 0.572309046885318.3%


In [5]:
results = []
for _, row in amt_of_sequences.iterrows():
    n = row["count"]
    x = row["sum"]
    if n < 15:
        continue
    stat, pval = proportions_ztest([x, total_success], [n, total_contacts])
    results.append({
        "Sequences": row["Number of Sequences Enrolled"],
        "Rate": row["Rate"] * 100,
        "Sample Size": n,
        "p-value": pval,
        "Significant": pval < 0.05
    })
sig_df = pd.DataFrame(results)
display(sig_df)

,Sequences,Rate,Sample Size,p-value,Significant
0,1.0,34.650035,1443.0,0.297830,False
1,2.0,41.493776,723.0,0.595358,False
2,3.0,114.503817,524.0,0.117100,False
3,4.0,40.160643,498.0,0.626554,False
4,5.0,88.300221,453.0,0.414383,False
5,6.0,34.722222,288.0,0.619273,False
6,7.0,45.248869,221.0,0.816761,False
7,8.0,198.675497,151.0,0.029095,True
8,9.0,0.000000,86.0,0.481720,False
9,10.0,0.000000,65.0,0.540771,False


In [16]:
# Check significance based on number of sales activities
# Results - between 30 - 70 touches were statiscally effective

amt_of_activities = (
    df.groupby("Number of Sales Activities")["Meeting Booked"]
        .agg(["count", "sum"])
        .reset_index()
    )
amt_of_activities["Rate"] = amt_of_activities["sum"] / amt_of_activities["count"] * 100

total_success = (df["Meeting Booked"] != 0).sum()
total_contacts = len(df)
overall_rate = total_success / total_contacts * 100
print(f"Overall booking rate: {overall_rate}.3%")

Overall booking rate: 0.572309046885318.3%


In [7]:
results = []
for _, row in amt_of_activities.iterrows():
    n = row["count"]
    x = row["sum"]
    if n < 15:
        continue
    stat, pval = proportions_ztest([x, total_success], [n, total_contacts])
    results.append({
        "Sales Activities": row["Number of Sales Activities"],
        "Rate": row["Rate"] * 100,
        "Sample Size": n,
        "p-value": pval,
        "Significant": pval < 0.05
    })
sig_df = pd.DataFrame(results)
display(sig_df[sig_df["Significant"] != False])

,Sales Activities,Rate,Sample Size,p-value,Significant
32,33.0,333.333333,60.0,0.006285,True
36,37.0,363.636364,55.0,0.003692,True
39,40.0,465.116279,43.0,0.000632,True
49,50.0,344.827586,29.0,0.043913,True
63,64.0,588.235294,17.0,0.004394,True
64,65.0,555.555556,18.0,0.005949,True
66,67.0,588.235294,17.0,0.004394,True
68,69.0,476.190476,21.0,0.012496,True


In [23]:
# Check significance based solely on role category (job level)
# Result - not statistically significant

df["role_category"] = df.apply(lambda row: row_to_prospect(row).get_role_category(), axis=1)
summary = (
    df.groupby('role_category')["Meeting Booked"]
    .agg(["count", 'sum'])
    .reset_index()
)

summary["Rate"] = summary["sum"] / summary["count"] * 100

summary = summary[summary["count"] >= 20]

total_success = (df["Meeting Booked"] != 0).sum()
total_contacts = len(df)
overall_rate = total_success / total_contacts

results = []
for _, row in summary.iterrows():
    n = row["count"]
    x = row["sum"]

    stat, pval = proportions_ztest([x, total_success], [n, total_contacts])
    results.append({
        "Role Category": row["role_category"],        
        "Rate": row["Rate"] * 100,
        "Sample Size": n,
        "p-value": pval,
        "Significant": pval < 0.05
    })
sig_df = pd.DataFrame(results)
display(sig_df)

,Role Category,Rate,Sample Size,p-value,Significant
0,Director,81.012658,1975,0.270135,False
1,Executive Director,0.000000,109,0.428339,False
2,Head,0.000000,121,0.404004,False
3,Manager,22.371365,894,0.183163,False
4,Vice President,66.603235,1051,0.720673,False
5,other,29.850746,335,0.514493,False


In [24]:
# Check based on functional group
# Results - workplace and capital/campus planning good, no discernable functional group is bad

df["func_group"] = df.apply(lambda row: row_to_prospect(row).get_functional_group(), axis=1)
summary = (
    df.groupby(['role_category', "func_group"])["Meeting Booked"]
    .agg(["count", 'sum'])
    .reset_index()
)

summary["Rate"] = summary["sum"] / summary["count"] * 100

summary = summary[summary["count"] >= 20]

total_success = (df["Meeting Booked"] != 0).sum()
total_contacts = len(df)
overall_rate = total_success / total_contacts

results = []
for _, row in summary.iterrows():
    n = row["count"]
    x = row["sum"]

    stat, pval = proportions_ztest([x, total_success], [n, total_contacts])
    results.append({
        "Functional Group": row["func_group"],        
        "Rate": row["Rate"] * 100,
        "Sample Size": n,
        "p-value": pval,
        "Significant": pval < 0.05
    })
sig_df = pd.DataFrame(results)
display(sig_df[sig_df["Significant"] == True])

,Functional Group,Rate,Sample Size,p-value,Significant
0,Campus/Capital,263.157895,76,0.021809,True
5,Workplace,203.252033,246,0.005410,True
11,Other,416.666667,24,0.021970,True


In [19]:
# Check significance based on title and function (together)
# Result:
    # Director of Campus/Capital Planning and Director of Workplace
    # Manager of Other was statistically worse off

df["role_category"] = df.apply(lambda row: row_to_prospect(row).get_role_category(), axis=1)
df["func_group"] = df.apply(lambda row: row_to_prospect(row).get_functional_group(), axis=1)
summary = (
    df.groupby(['role_category', "func_group"])["Meeting Booked"]
    .agg(["count", 'sum'])
    .reset_index()
)

summary["Rate"] = summary["sum"] / summary["count"] * 100

summary = summary[summary["count"] >= 20]

total_success = (df["Meeting Booked"] != 0).sum()
total_contacts = len(df)
overall_rate = total_success / total_contacts

results = []
for _, row in summary.iterrows():
    n = row["count"]
    x = row["sum"]

    stat, pval = proportions_ztest([x, total_success], [n, total_contacts])
    results.append({
        "Role Category": row["role_category"],
        "Functional Group": row["func_group"],        
        "Rate": row["Rate"] * 100,
        "Sample Size": n,
        "p-value": pval,
        "Significant": pval < 0.05
    })
sig_df = pd.DataFrame(results)
display(sig_df)

,Role Category,Functional Group,Rate,Sample Size,p-value,Significant
0,Director,Campus/Capital,263.157895,76,0.021809,True
1,Director,Facilities,64.516129,930,0.790629,False
2,Director,Other,0.000000,85,0.484279,False
3,Director,Real Estate,35.650624,561,0.513841,False
4,Director,Space,140.845070,71,0.359360,False
5,Director,Workplace,203.252033,246,0.005410,True
6,Executive Director,Facilities,0.000000,54,0.577187,False
7,Executive Director,Real Estate,0.000000,23,0.715972,False
8,Head,Facilities,0.000000,29,0.682865,False
9,Head,Real Estate,0.000000,71,0.522660,False


In [12]:
# Check significance from sequences
# Result - Campus/Capital Planning was significantly more effective
summary = (
    df.groupby("Last Sequence Enrolled")["Meeting Booked"]
    .agg(["count", 'sum'])
    .reset_index()
)

summary["Rate"] = summary["sum"] / summary["count"] * 100

summary = summary[summary["count"] >= 20]

total_success = (df["Meeting Booked"] != 0).sum()
total_contacts = len(df)
overall_rate = total_success / total_contacts

results = []
for _, row in summary.iterrows():
    n = row["count"]
    x = row["sum"]

    stat, pval = proportions_ztest([x, total_success], [n, total_contacts])
    results.append({
        "Last Sequence Enrolled": row["Last Sequence Enrolled"],        
        "Rate": row["Rate"] * 100,
        "Sample Size": n,
        "p-value": pval,
        "Significant": pval < 0.05
    })
sig_df = pd.DataFrame(results)
display(sig_df)

,Last Sequence Enrolled,Rate,Sample Size,p-value,Significant
0,269580465.0,85.348506,703.0,0.372926,False
1,269668391.0,38.147139,1835.0,0.336219,False
2,279109668.0,57.088487,1051.0,0.995598,False
3,279559810.0,185.873606,269.0,0.010394,True
4,279606875.0,0.000000,104.0,0.439132,False
5,282518921.0,34.423408,581.0,0.482567,False
